In [4]:
from dotenv import load_dotenv

load_dotenv("openai.env")

True

# LCEL:memory添加方式
<hr>

In [9]:
from operator import itemgetter

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个乐于助人的机器人"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
memory = ConversationBufferMemory(return_messages=True)
memory.load_memory_variables({})

{'history': []}

定义一条链

In [11]:
chain = (
        RunnablePassthrough.assign(
            history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
        )
        | prompt
        | model
)
inputs = {"input": "你好我是tom"}
response = chain.invoke(inputs)
print(response)

content='你好，Tom！有什么问题我可以帮助你解决吗？' response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 29, 'total_tokens': 52}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None} id='run-aaccd67c-b57e-4ef7-8a89-2c432e94bfeb-0'


# 手动保存记忆

In [12]:
#保存记忆
memory.save_context(inputs, {"output": response.content})
memory.load_memory_variables({})

{'history': [HumanMessage(content='你好我是tom'),
  AIMessage(content='你好，Tom！有什么问题我可以帮助你解决吗？')]}

In [13]:
response = chain.invoke({"input": "我叫什么名字?"})
response

AIMessage(content='你告诉我你的名字是Tom。', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 69, 'total_tokens': 81}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-d3704fd5-23eb-4c28-afee-5482d895467d-0')

# 使用Redis来实现长时记忆

In [14]:
! pip install redis

Looking in indexes: http://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 2.0 MB/s eta 0:00:0000:0100:01


In [16]:
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个擅长{ability}的助手"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatOpenAI(model="gpt-4-1106-preview", temperature=0)

chain_with_history = RunnableWithMessageHistory(
    chain,
    #使用redis存储聊天记录
    lambda session_id: RedisChatMessageHistory(session_id, url="redis://localhost:6379/0"),
    input_messages_key="question",
    history_messages_key="history",
)

In [19]:
#每次调用都会保存聊天记录，需要有对应的session_id
chain_with_history.invoke(
    {"ability": "历史", "question": "中国建都时间最长的城市是哪个?"},
    config={"configurable": {"session_id": "jh-1992"}},
)

AIMessage(content='中国历史上建都时间最长的城市是西安。西安古称长安，是中国古代历史上著名的都城之一，曾是周、秦、汉、隋、唐等多个朝代的都城。特别是在唐朝时期，长安成为当时世界上最繁华的城市之一。从周朝的西周时期开始，西安作为都城的历史持续了超过一千年，是中国乃至世界历史上最重要的文化都市之一。', response_metadata={'token_usage': {'completion_tokens': 157, 'prompt_tokens': 39, 'total_tokens': 196}, 'model_name': 'gpt-4-1106-preview', 'system_fingerprint': 'fp_d2051b8491', 'finish_reason': 'stop', 'logprobs': None}, id='run-26707d12-3849-4ef0-8ba4-17f12f059e86-0')

In [20]:
chain_with_history.invoke(
    {"ability": "历史", "question": "它有多少年建都历史？"},
    config={"configurable": {"session_id": "jh-1992"}},
)

AIMessage(content='西安作为都城的历史非常悠久，但具体的年数需要根据不同朝代的建都时间来计算。下面是西安（长安）作为都城的主要历史时期：\n\n1. 西周（前1046年 - 前771年）：西周时期，周王朝的都城设在镐京（今西安西北部的周原），这标志着西安作为都城的开始。\n2. 秦朝（前221年 - 前206年）：秦始皇统一六国后，将都城定在咸阳（今西安市西北），而长安（今西安市区）则是秦朝的一个重要城市。\n3. 西汉（前202年 - 9年）：西汉初年，汉高祖刘邦定都长安。\n4. 新朝（9年 - 23年）：王莽篡汉后，改长安为“常安”。\n5. 东汉（25年 - 220年）：东汉时期，都城迁至洛阳，但长安依然是重要城市。\n6. 西晋（265年 - 316年）：晋武帝司马炎迁都长安，但不久后因为“永嘉之乱”而失去对长安的控制。\n7. 前赵（304年 - 329年）：汉赵时期，刘渊曾短暂定都长安。\n8. 前秦（351年 - 394年）：苻坚定都长安，并在此建立了较为强盛的政权。\n9. 隋朝（581年 - 618年）：隋文帝杨坚统一中国后，定都大兴城（即隋炀帝时期改称为长安）。\n10. 唐朝（618年 - 907年）：唐朝继续以长安为都，这一时期的长安是世界上最繁华的城市之一。\n\n如果将上述时期作为都城的时间累加起来，西安作为都城的时间大约有1200年左右。需要注意的是，这里的计算并没有包括一些短暂或不连续的建都时期，而且在不同朝代间的过渡期，都城的地位可能会有所变动。因此，上述年数仅供参考。', response_metadata={'token_usage': {'completion_tokens': 663, 'prompt_tokens': 216, 'total_tokens': 879}, 'model_name': 'gpt-4-1106-preview', 'system_fingerprint': 'fp_2f57f81c11', 'finish_reason': 'stop', 'logprobs': None}, id='run-cfc82f6b-590c-4ef3-8a70-b533eabc16d1-0')